In [1]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

from test_tool import multiply, load_cv_info

# Create the agent
memory = MemorySaver()
model = init_chat_model("openai:gpt-4")
#search = TavilySearch(max_results=2)
tools = [multiply, load_cv_info]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [2]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Provide CV feedback",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Provide CV feedback
================================== Ai Message ==================================

Sure, but first I need to check the content of your CV. Please allow me to load it.
Tool Calls:
  load_cv_info (call_Lczwq15UJSm7ew5rZPwEQUAC)
 Call ID: call_Lczwq15UJSm7ew5rZPwEQUAC
  Args:
================================= Tool Message =================================
Name: load_cv_info

Fergal Riordan
Ireland (Willing to Relocate) | +353 838754381 | fergalriordan333@gmail.com | Portfolio Website | GitHub | LinkedIn


PROFILE	Software Engineer with a First-Class Honours MEng in Electronic & Computer Engineering from Trinity College Dublin. Experience spans Agentic AI development for a Partner Relationship Management (PRM) platform, RLHF data annotation for LLMs, and generative AI research in CycleGAN architectures. Proficient in Python and PyTorch, with a passion for building responsible, human-centred